In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

In [5]:
# data = pd.read_csv("synthetic_dataset_V2_10000.csv")
df = pd.read_csv('../datasets/finaltraining_data.csv')

In [6]:
df.head(5)

,Ticket ID,Customer Query,Summary,Assigned Department,Action Type,Priority,Sentiment,Timestamp,Resolution_Steps,Resolution_Time_Actual,Complexity_Score
0,T18644,Do you offer services in the London region or ...,Do you offer services in the London region or ...,General Queries,Follow-Up,Low,Neutral,09-06-2024 03:59,Forwarded pre-sales inquiry to a sales represe...,864,5
1,T17866,Do you offer services in the Paris region or o...,Do you offer services in the Paris region or o...,General Queries,Escalate,Low,Neutral,28-12-2024 13:54,Customer is a high-value lead demanding immedi...,1153,5
2,T14525,I appreciate the quick resolution on my invoic...,I appreciate the quick resolution on my invoic...,Billing,Resolve at L1,Low,Positive,20-12-2024 16:16,Checked plan details and billing cycle. Explai...,1033,5
3,T15260,The mobile app crashes every time I try to ope...,The mobile app crashes every time I try to ope...,App Support,Escalate,High,Negative,16-12-2024 14:41,App is crashing universally after latest relea...,386,9
4,T15887,I am constantly getting a 'server connection f...,I am constantly getting a 'server connection f...,Technical Support,Follow-Up,Medium,Negative,26-06-2024 20:14,Identified potential bug with latest patch. Lo...,399,9


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6600 entries, 0 to 6599
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Ticket ID               6600 non-null   object
 1   Customer Query          6600 non-null   object
 2   Summary                 6600 non-null   object
 3   Assigned Department     6600 non-null   object
 4   Action Type             6600 non-null   object
 5   Priority                6600 non-null   object
 6   Sentiment               6600 non-null   object
 7   Timestamp               6600 non-null   object
 8   Resolution_Steps        6600 non-null   object
 9   Resolution_Time_Actual  6600 non-null   int64 
 10  Complexity_Score        6600 non-null   int64 
dtypes: int64(2), object(9)
memory usage: 567.3+ KB


In [ ]:
data['Sentiment'].value_counts()

Sentiment
Negative    4332
Neutral     3173
Positive    2495
Name: count, dtype: int64

In [ ]:
data['Complexity_Score'].size

10000

In [ ]:
mask = (data['Complexity_Score'] > 8) & (data['Sentiment'] == 'Negative')
target_count = 1100
current_count = mask.sum()

if current_count < target_count:
    needed = target_count - current_count
    potential_indices = data[~mask].index
    chosen_indices = np.random.choice(potential_indices, needed, replace=False)
    
    # Force these records to meet the condition
    data.loc[chosen_indices, 'Complexity_Score'] = np.random.uniform(8.1, 10.0, needed)
    data.loc[chosen_indices, 'Sentiment'] = 'Negative'


In [ ]:
data.loc[mask, 'Resolution_Time_Actual'] = np.random.randint(300, 501, size=current_count)

In [ ]:
data_modified = data.drop(columns=['Complexity_Score'])

In [ ]:
data_modified.to_csv('modified.csv',index=False)

### Balancing action type

In [ ]:
data = pd.read_csv("modified.csv")

In [ ]:
# 6,600 total dataset
target_n = 2200

# 1. Separate by Action Type (Our main target)
data_l1 = data[data['Action Type'] == 'Resolve at L1']
data_fup = data[data['Action Type'] == 'Follow-Up']
data_esc = data[data['Action Type'] == 'Escalate']

In [ ]:
# 2. Downsample/Upsample to make them equal
data_l1_final = resample(data_l1, replace=False, n_samples=target_n, random_state=42)
data_fup_final = resample(data_fup, replace=False, n_samples=target_n, random_state=42)
data_esc_final = resample(data_esc, replace=True, n_samples=target_n, random_state=42)

In [ ]:
action_balanced = pd.concat([data_l1_final, data_fup_final, data_esc_final]).sample(frac=1).reset_index(drop=True)

In [ ]:

print(action_balanced.shape)
print(action_balanced['Action Type'].value_counts())

(6600, 10)
Action Type
Escalate         2200
Resolve at L1    2200
Follow-Up        2200
Name: count, dtype: int64


In [ ]:
mod = action_balanced.to_csv('balanced.csv',index=False)

In [ ]:
data.columns

Index(['Ticket ID', 'Customer Query', 'Summary', 'Assigned Department',
       'Action Type', 'Priority', 'Sentiment', 'Timestamp', 'Resolution_Steps',
       'Resolution_Time_Actual'],
      dtype='object')

### Balancing Priority type

In [ ]:
data = pd.read_csv("balanced.csv")

In [ ]:
print(data['Priority'].value_counts())

Priority
Low       2571
Medium    2399
High      1630
Name: count, dtype: int64


In [ ]:
data_low = data[data['Priority'] == 'Low']
data_medium = data[data['Priority'] == 'Medium']
data_high = data[data['Priority'] == 'High']

In [ ]:
need = 2400 # for low and medium
need1 = 1800 # for high


In [ ]:



data_medium_ = resample(data_medium, 
                               replace=True,     
                               n_samples=need, 
                               random_state=42) 


data_high_ = resample(data_high, 
                             replace=True,    
                             n_samples=need1, 
                             random_state=42)


data_low_ = resample(data_low, 
                             replace=True,    
                             n_samples=need, 
                             random_state=42)


In [ ]:
priority_balanced = pd.concat([data_low_, data_high_, data_medium_]).sample(frac=1).reset_index(drop=True)
# print(priority_balanced.value_counts())


In [ ]:
priority_balanced['Priority'].value_counts()

Priority
Low       2400
Medium    2400
High      1800
Name: count, dtype: int64

### Balancing Sentiment type

In [ ]:
priority_balanced['Sentiment'].value_counts()

Sentiment
Negative    3463
Neutral     1811
Positive    1326
Name: count, dtype: int64

In [ ]:
neu = 2100
pos = 1700
neg = 2800

In [ ]:
data_neg = data[data['Sentiment'] == 'Negative']
data_neu = data[data['Sentiment'] == 'Neutral']
data_pos = data[data['Sentiment'] == 'Positive']

In [ ]:

data_neg_ = resample(data_neg, 
                               replace=False,     
                               n_samples=neg, 
                               random_state=42) 


data_pos_ = resample(data_pos, 
                             replace=True,    
                             n_samples=pos, 
                             random_state=42)


data_neu_ = resample(data_neu, 
                             replace=True,    
                             n_samples=neu, 
                             random_state=42)

In [ ]:
sen_balanced = pd.concat([data_neg_, data_neu_, data_pos_]).sample(frac=1).reset_index(drop=True)


In [ ]:
sen_balanced['Sentiment'].value_counts()

Sentiment
Negative    2800
Neutral     2100
Positive    1700
Name: count, dtype: int64

In [ ]:
sen_balanced['Resolution_Time_Actual'].head()

0     63
1    141
2     31
3    146
4     69
Name: Resolution_Time_Actual, dtype: int64

In [ ]:
sen_balanced.to_csv('balanced.csv',index=False)

### SLA and resolution time optimization

In [ ]:
data = pd.read_csv('balanced.csv')

In [ ]:
def balance_res(row):
    # 1. Create "The Critical Breach"
    # 15% of High Priority/Negative tickets will be set to fail the SLA (300-500 mins)
    if row['Priority'] == 'High' and row['Sentiment'] == 'Negative' and np.random.rand() < 0.15:
        return np.random.randint(250, 501)
    
    # b. Normal Priority-Based Queue Logic

    # High (Fast response)
    if row['Priority'] == 'High':
        return np.random.randint(15, 91)
    # Medium (Moderate queue)
    elif row['Priority'] == 'Medium':
        return np.random.randint(120, 301)
    # Low: (Long queue, but still under 1080m SLA)
    else:
        return np.random.randint(400, 801)


In [ ]:

data['Resolution_Time_Actual'] = data.apply(balance_res, axis=1)

In [ ]:
data['Resolution_Time_Actual']

<bound method Series.info of 0       535
1       567
2       667
3       347
4       186
       ... 
6595    751
6596    122
6597    239
6598    645
6599    277
Name: Resolution_Time_Actual, Length: 6600, dtype: int64>

In [ ]:
print(data.groupby('Priority')['Resolution_Time_Actual'].agg(['mean', 'max']))

                mean  max
Priority                 
High       79.667732  497
Low       599.243346  800
Medium    209.126403  300


### Creating Delays

In [ ]:
def apply_delay(row):
    # 1. Base_work(min time)
    base_work = row['Resolution_Time_Actual']
    
    # High Priority: Fast (0-40 min delay)
    if row['Priority'] == 'High':
        # INJECT BREACHES: 15% of High-Neg tickets fail the 240m SLA
        if row['Sentiment'] == 'Negative' and np.random.rand() < 0.15:
            return np.random.randint(250, 500) # 'Escalation' triggers
        return base_work + np.random.randint(0, 41) ## normal ticket (unbreached)
    
    # Medium Priority: Moderate Queue (120-240 min delay)
    elif row['Priority'] == 'Medium':
        return base_work + np.random.randint(120, 241)
    
    # Low Priority: Long Queue (300-600 min delay)
    else:
        return base_work + np.random.randint(300, 601)

In [ ]:
data['Resolution_Time_Actual'] = data.apply(apply_delay, axis=1)

In [ ]:
print(data.groupby('Priority')['Resolution_Time_Actual'].agg(['mean', 'max']))

                 mean   max
Priority                   
High       120.079872   523
Low       1051.142205  1384
Medium     389.610811   537


In [ ]:
## tickets to be detected by agent(breached)
tic = len(data[(data['Priority'] == "High") & (data['Sentiment'] == "Negative") & (data['Resolution_Time_Actual'] > 240)])

In [ ]:
tic

242

In [ ]:
data.to_csv('balanced.csv',index=False)

In [ ]:
data.head()

,Ticket ID,Customer Query,Summary,Assigned Department,Action Type,Priority,Sentiment,Timestamp,Resolution_Steps,Resolution_Time_Actual
0,T18644,Do you offer services in the London region or ...,Do you offer services in the London region or ...,General Queries,Follow-Up,Low,Neutral,09-06-2024 03:59,Forwarded pre-sales inquiry to a sales represe...,864
1,T17866,Do you offer services in the Paris region or o...,Do you offer services in the Paris region or o...,General Queries,Escalate,Low,Neutral,28-12-2024 13:54,Customer is a high-value lead demanding immedi...,1153
2,T14525,I appreciate the quick resolution on my invoic...,I appreciate the quick resolution on my invoic...,Billing,Resolve at L1,Low,Positive,20-12-2024 16:16,Checked plan details and billing cycle. Explai...,1033
3,T15260,The mobile app crashes every time I try to ope...,The mobile app crashes every time I try to ope...,App Support,Escalate,High,Negative,16-12-2024 14:41,App is crashing universally after latest relea...,386
4,T15887,I am constantly getting a 'server connection f...,I am constantly getting a 'server connection f...,Technical Support,Follow-Up,Medium,Negative,26-06-2024 20:14,Identified potential bug with latest patch. Lo...,399


: 